### Carregar pesos de GPT2 Para modelo scratch

O objetivo deste notebook é carregar os pesos fornecidos pelo modelo pré-treinado GPT2 a uma instância de um modelo baseado no GPT criado do 0.

Iremos utilizar os códigos previamente construídos nos capítulos anteriores do livro Build a Large Language Model (From Scratch), onde aqui serão apresentados apenas os códigos referentes a seção 5.5.

In [4]:
# !pip install tiktoken
# !pip install tqdm

In [ ]:
import torch
import tiktoken
import numpy as np


# A seguir temos códigos auxiliares fornecidos em sessões anteriores do livro
# Código para baixar pesos fornecidos pela OpenAI
from gpt_download import download_and_load_gpt2 

# Código que contém a estrutura necessária do GPT-2 sem pesos
from gpt_structure import GPTModel

# Código com funções de texto
from gpt_text_functions import text_to_token_ids, token_ids_to_text, generate 



Vamos baixar os pesos existentes do GPT2 para o modelo de 124M de parâmetros.

In [ ]:
settings, params = download_and_load_gpt2(
    model_size="124M", models_dir="gpt2"
)

checkpoint: 100%|██████████| 77.0/77.0 [00:00<00:00, 115kiB/s]
encoder.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 2.95MiB/s]
hparams.json: 100%|██████████| 90.0/90.0 [00:00<00:00, 170kiB/s]
model.ckpt.data-00000-of-00001: 100%|██████████| 498M/498M [00:44<00:00, 11.1MiB/s]
model.ckpt.index: 100%|██████████| 5.21k/5.21k [00:00<00:00, 10.9MiB/s]
model.ckpt.meta: 100%|██████████| 471k/471k [00:00<00:00, 1.43MiB/s]
vocab.bpe: 100%|██████████| 456k/456k [00:00<00:00, 1.92MiB/s]


In [ ]:
print("Configurações:", settings)
print("Chaves com o dicionário de parâmetros:", params.keys())

Settings: {'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 768, 'n_head': 12, 'n_layer': 12}
Parameter dictionary keys: dict_keys(['blocks', 'b', 'g', 'wpe', 'wte'])


In [ ]:
print(params["wte"])
print("Dimensão dos tensores dos embeddings de token:", params["wte"].shape)

[[-0.11010301 -0.03926672  0.03310751 ... -0.1363697   0.01506208
   0.04531523]
 [ 0.04034033 -0.04861503  0.04624869 ...  0.08605453  0.00253983
   0.04318958]
 [-0.12746179  0.04793796  0.18410145 ...  0.08991534 -0.12972379
  -0.08785918]
 ...
 [-0.04453601 -0.05483596  0.01225674 ...  0.10435229  0.09783269
  -0.06952604]
 [ 0.1860082   0.01665728  0.04611587 ... -0.09625227  0.07847701
  -0.02245961]
 [ 0.05135201 -0.02768905  0.0499369  ...  0.00704835  0.15519823
   0.12067825]]
Token embedding weight tensor dimensions: (50257, 768)


Devemos atualizar as configurações da nossa estrutura do modelo a ser criado para garantir a correspondência dos tamanhos de token e embedding.

In [ ]:
# Vamos utilizar apenas a configuração do 124M

GPT_CONFIG_124M = {
 "vocab_size": 50257,
 "context_length": 1024, # Aderência ao tamanho da OpenAI
 "emb_dim": 768,
 "n_heads": 12,
 "n_layers": 12,
 "drop_rate": 0.1, 
 "qkv_bias": True # OpenAI usa vetor de Bias (manter consistência)
}

In [ ]:
torch.manual_seed(42)
model = GPTModel(GPT_CONFIG_124M) # Criando modelo do scratch com os parâmetros de 124M
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features

Checa se dois tensores ou arrays possuem a mesma dimensão para retornar o tensor que sobrescreverá os pesos do modelo.

In [ ]:
def assign(left, right):
  if left.shape != right.shape:
      raise ValueError(f"Diferença de tamanhos. Left: {left.shape}, "
      "Right: {right.shape}"
      )
  return torch.nn.Parameter(torch.tensor(right))

A próxima função irá ajustar todos os tipos de pesos existentes ao nosso modelo recém criado.

In [ ]:
def load_weights_into_gpt(gpt, params):
  gpt.pos_emb.weight = assign(gpt.pos_emb.weight, params['wpe'])
  gpt.tok_emb.weight = assign(gpt.tok_emb.weight, params['wte'])

  for b in range(len(params["blocks"])):
    q_w, k_w, v_w = np.split(
      (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
    gpt.trf_blocks[b].att.W_query.weight = assign(
      gpt.trf_blocks[b].att.W_query.weight, q_w.T)
    gpt.trf_blocks[b].att.W_key.weight = assign(
      gpt.trf_blocks[b].att.W_key.weight, k_w.T)
    gpt.trf_blocks[b].att.W_value.weight = assign(
      gpt.trf_blocks[b].att.W_value.weight, v_w.T)

    q_b, k_b, v_b = np.split(
      (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
    gpt.trf_blocks[b].att.W_query.bias = assign(
      gpt.trf_blocks[b].att.W_query.bias, q_b)
    gpt.trf_blocks[b].att.W_key.bias = assign(
      gpt.trf_blocks[b].att.W_key.bias, k_b)
    gpt.trf_blocks[b].att.W_value.bias = assign(
      gpt.trf_blocks[b].att.W_value.bias, v_b)

    gpt.trf_blocks[b].att.out_proj.weight = assign(
      gpt.trf_blocks[b].att.out_proj.weight,
      params["blocks"][b]["attn"]["c_proj"]["w"].T)
    gpt.trf_blocks[b].att.out_proj.bias = assign(
      gpt.trf_blocks[b].att.out_proj.bias,
      params["blocks"][b]["attn"]["c_proj"]["b"])
    gpt.trf_blocks[b].ff.layers[0].weight = assign(
      gpt.trf_blocks[b].ff.layers[0].weight,
      params["blocks"][b]["mlp"]["c_fc"]["w"].T)
    gpt.trf_blocks[b].ff.layers[0].bias = assign(
      gpt.trf_blocks[b].ff.layers[0].bias,
      params["blocks"][b]["mlp"]["c_fc"]["b"])
    gpt.trf_blocks[b].ff.layers[2].weight = assign(
      gpt.trf_blocks[b].ff.layers[2].weight,
      params["blocks"][b]["mlp"]["c_proj"]["w"].T)
    gpt.trf_blocks[b].ff.layers[2].bias = assign(
      gpt.trf_blocks[b].ff.layers[2].bias,
      params["blocks"][b]["mlp"]["c_proj"]["b"])

    gpt.trf_blocks[b].norm1.scale = assign(
      gpt.trf_blocks[b].norm1.scale,
      params["blocks"][b]["ln_1"]["g"])
    gpt.trf_blocks[b].norm1.shift = assign(
      gpt.trf_blocks[b].norm1.shift,
      params["blocks"][b]["ln_1"]["b"])
    gpt.trf_blocks[b].norm2.scale = assign(
      gpt.trf_blocks[b].norm2.scale,
      params["blocks"][b]["ln_2"]["g"])
    gpt.trf_blocks[b].norm2.shift = assign(
      gpt.trf_blocks[b].norm2.shift,
      params["blocks"][b]["ln_2"]["b"])

  gpt.final_norm.scale = assign(gpt.final_norm.scale, params["g"])
  gpt.final_norm.shift = assign(gpt.final_norm.shift, params["b"])
  gpt.out_head.weight = assign(gpt.out_head.weight, params["wte"])

Verifica a existência de GPU para utilização.

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Chama a transferência de pesos para o novo modelo criado. Params provém do download inicial que fizemos dos pesos da OpenAI.

In [ ]:
load_weights_into_gpt(model, params)
model.to(device)

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(1024, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=True)
        (W_key): Linear(in_features=768, out_features=768, bias=True)
        (W_value): Linear(in_features=768, out_features=768, bias=True)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (norm1): LayerNorm()
      (norm2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=7

### Geração de texto

Vamos utilizar o tokenizador do tiktoken para o GPT-2.

In [28]:
tokenizer = tiktoken.get_encoding("gpt2")

In [ ]:
torch.manual_seed(42)

token_ids = generate(
  model=model,
  idx=text_to_token_ids("Eu sou um cientista de", tokenizer).to(device),
  max_new_tokens=40,
  context_size=GPT_CONFIG_124M["context_length"],
  top_k=50,
  temperature=1.2
)
print("Texto de saída:\n", token_ids_to_text(token_ids, tokenizer))


Output text:
 Every effort moves you as far as the hand can go until the end of your turn unless something happens

This would remove you from a battle
